In [1]:
import numpy as np
import os, sys
from datetime import datetime
import requests
from requests.exceptions import HTTPError
%load_ext autoreload
%autoreload 2

In [2]:
HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(DIR)
from utilities.model.center_of_mass import CenterOfMass
from utilities.model.structure import Structure
from sqlalchemy import func
from sql_setup import session

In [11]:
def get_transformation_matrix(animal):
    try:
        #url = f'https://activebrainatlas.ucsd.edu/activebrainatlas/rotation/{animal}/manual/2'
        url = f'http://localhost:8000/rotation/{animal}/manual/2'
        response = requests.get(url)
        response.raise_for_status()
        # access JSOn content
        transformation_matrix = response.json()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

    return transformation_matrix

In [22]:
prep_id = 'DK39'

In [23]:
transformation = get_transformation_matrix(prep_id)
r = np.array(transformation['rotation'])
t = np.array(transformation['translation'])

In [24]:
print(r)
print(t)

[[ 0.9072506   0.01008301 -0.01509823]
 [-0.00758282  0.8960958   0.14278686]
 [ 0.01649619 -0.14263211  0.89600069]]
[[-620.2284516 ]
 [-120.71434773]
 [ -39.55153204]]


In [25]:
# person_id = 2 = beth
rows = session.query(CenterOfMass).filter(
    CenterOfMass.active.is_(True))\
        .filter(CenterOfMass.prep_id == prep_id)\
        .filter(CenterOfMass.person_id == 2)\
        .filter(CenterOfMass.input_type == 'manual')\
        .all()
row_dict = {}
for row in rows:
    structure = row.structure.abbreviation
    row_dict[structure] = [row.x, row.y, row.section]

In [26]:
def brain_to_atlas_transform(
    brain_coord, r, t,
    brain_scale=(0.325, 0.325, 20),
    atlas_scale=(10, 10, 20)
):
    """
    Takes an x,y,z brain coordinates, and a rotation matrix and transform vector.
    Returns the point in atlas coordinates.
    
    The recorded r, t is the transformation from brain to atlas such that:
        t_phys = atlas_scale @ t
        atlas_coord_phys = r @ brain_coord_phys + t_phys
        
    Currently we erraneously have:
        t_phys = brain_scale @ t
    This should be fixed in the future.
    """
    brain_scale = np.diag(brain_scale)
    atlas_scale = np.diag(atlas_scale)

    # Transform brain coordinates to physical space
    brain_coord = np.array(brain_coord).reshape(3, 1) # Convert to a column vector
    brain_coord_phys = brain_scale @ brain_coord
    
    # Apply affine transformation in physical space
    # t_phys = atlas_scale @ t
    # The following is wrong but it is what the rest of the code assumes
    # We need to fix it in the future
    t_phys = brain_scale @ t
    atlas_coord_phys = r @ brain_coord_phys + t_phys

    # Bring atlas coordinates back to atlas space
    atlas_coord = np.linalg.inv(atlas_scale) @ atlas_coord_phys

    return atlas_coord.T[0] # Convert back to a row vector

In [27]:
data = []
scales = np.diag([32,32,1])
person_id = 28 # bili, this is your database ID
for abbrev,v in row_dict.items():
    x = v[0]
    y = v[1]
    section = v[2]
    structure = session.query(Structure).filter(Structure.abbreviation == func.binary(abbrev)).one()
    data.append([x, y, section])
    brain_coords = np.asarray([x, y, section])
    transformed = brain_to_atlas_transform(brain_coords, r, t)
    print(abbrev, x, y, section, transformed @ scales)
    continue
    com = CenterOfMass(
        prep_id=prep_id, structure=structure, x=x, y=y, section=section,
        created=datetime.utcnow(), active=True, person_id=person_id, input_type='aligned'
    )
    try:
        session.add(com)
        session.commit()
    except Exception as e:
        print(f'No merge for {abbrev} {e}')
        session.rollback()

data = np.array(data)

10N_L 41901.0 22575.0 198.0 [38935.66228166 22391.95266309    96.76501864]
10N_R 43814.0 18639.0 273.0 [40626.90980987 19394.12902556   173.60062479]
3N_L 33190.0 15847.0 233.0 [30612.10784985 16510.40145389   141.38391343]
3N_R 33423.0 15830.0 247.0 [30818.24653741 16620.65803861   154.02978387]
5N_L 37069.0 20100.0 178.0 [34369.84604875 19940.74066948    83.28620907]
5N_R 36316.0 18831.0 324.0 [33504.97498655 20098.24795312   216.84171011]
6N_L 38545.0 19663.0 233.0 [35704.78372795 20024.45286937   133.97477428]
6N_R 38359.0 19339.0 272.0 [35488.20243    20080.36725162   169.61989936]
7N_L 39468.0 23531.0 203.0 [36645.22144689 23347.76570382    98.37703457]
7N_R 38803.0 22355.0 329.0 [35883.68291688 23408.48221271   213.82055866]
DC_L 40364.0 18199.0 140.0 [37495.5967691  17795.88098108    54.52753531]
DC_R 39577.0 15979.0 363.0 [36514.26876949 17771.03544894   259.27017603]
LC_L 38569.0 17311.0 203.0 [35731.75344342 17558.19081098   112.55258656]
LC_R 37587.0 16405.0 288.0 [34713.56

In [ ]:
# Print atlas COMs for comparison

from utilities.sqlcontroller import SqlController

def get_atlas_centers(
        atlas_box_size=(1000, 1000, 300),
        atlas_box_scales=(10, 10, 20),
        atlas_raw_scale=10
):
    atlas_box_scales = np.array(atlas_box_scales)
    atlas_box_size = np.array(atlas_box_size)
    atlas_box_center = atlas_box_size / 2
    sqlController = SqlController('Atlas')
    atlas_centers = sqlController.get_centers_dict('Atlas')

    for structure, center in atlas_centers.items():
        # transform into the atlas box coordinates that neuroglancer assumes
        center = atlas_box_center + np.array(center) * atlas_raw_scale / atlas_box_scales
        atlas_centers[structure] = center

    return atlas_centers

get_atlas_centers()